In [101]:
import datetime as dt
import math

import pandas as pd

from GoogleEarth import GoogleEarth

Define the gases and get the dispersion coefficient

In [106]:
# Data
df_gases = pd.DataFrame(columns=["Capacity", "Density", "Mass", "25M_Vol",
                                 "25Dispersion","100M_Mass", "100M_Vol","100Dispersion"],
                        index=["VC", "BE", "2-EHA", "IB", "BA", "BZ"])
df_gases["Capacity"] = [490000, 114000, 98000, 114000, 114000, 228000]  # The compressed volume in L
df_gases["Mass"] = [727560, 185750, 205900, 155642, 180000, 390000]
df_gases["Mass"] = df_gases["Mass"] * 0.453592 # The mass in kg
df_gases["Density"] = [0.911, 0.902, 0.875, 0.56, 0.902, 0.879]  # The normal density
df_gases["Vol"] = df_gases["Mass"] / df_gases["Density"] # The normal volume of gases at 25 degree in L
df_gases["N_Mass"] = [62.5, 118.17, 185.28, 56.11, 128.17, 78.11]  # Molar Mass in g/mol
df_gases["N"] = df_gases["Mass"] * 1000 / df_gases["N_Mass"] 

Find out the speed and dispersion coefficient that the gases are dispersed into air

In [104]:
# Assume everything is settle down when the volume is ideal
initial_mass = df_gases["Mass"].sum()
terminal_mass = df_gases["Vol"].sum() / 22.4 * (df_gases["Density"] * df_gases["Mass"] / df_gases["Mass"].sum()).sum()
dispersion_coeff = math.log(initial_mass, terminal_mass) / (24 * 3600)
dispersion_coeff # m^2/s, a reasonable number

1.4966967305001657e-05

In [107]:
source = (-80.522, 40.836)
start_time = dt.datetime(2023, 2, 3, 21)
ge = GoogleEarth()

In [110]:
df = ge.get_region_features([source[0]], [source[1]], ["wind_u", "wind_v", "temperature"], start_time, dt.timedelta(days=1))

In [112]:
dispersion_coeff * 3600

0.053881082298005964

In [111]:
df

,u,v,temperature,wind_u,wind_v
0,-80.522,40.836,-9.54,2.05875,0.696667
